In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U vaderSentiment

In [ ]:
# 1. Install STABLE versions to fix "cannot import name" errors
!pip install -q -U transformers==4.40.0 bitsandbytes==0.43.1 accelerate==0.29.3 peft==0.10.0 vaderSentiment

# 2. Login to Hugging Face (Paste your token below)
from huggingface_hub import login

# REPLACE 'hf_...' WITH YOUR ACTUAL TOKEN
# Do not leave spaces around the string
login(token="hugging face token")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 77.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.0 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [3]:
# Force install the absolute latest versions to support Gemma 2 and Mistral v0.3
!pip install -U "transformers>=4.43.0" "accelerate>=0.32.0" "bitsandbytes>=0.43.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 102.6 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1
  Attempting uninstall: accelerate
    Found exis

In [1]:
import torch
import pandas as pd
import csv
import gc
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, BitsAndBytesConfig
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# --- CONFIGURATION ---
MODELS_TO_TEST = [
    {"name": "Llama-3-8B", "id": "meta-llama/Meta-Llama-3-8B-Instruct", "type": "generative"},
    {"name": "Gemma-2-9B", "id": "google/gemma-2-9b-it", "type": "generative"},
    {"name": "Mistral-7B-v0.3", "id": "mistralai/Mistral-7B-Instruct-v0.3", "type": "generative"},
    {"name": "FinBERT", "id": "ProsusAI/finbert", "type": "bert"},
    {"name": "VADER", "id": "vader", "type": "vader"}
]

# --- UTILITY FUNCTIONS ---
def clean_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def predict_generative(text, model, tokenizer):
    text_input = str(text)[:512]
    prompt = (f"classify the sentiment of this piece of news headline: {text_input}. "
              f"Sentiment is 'positive', 'negative' or 'neutral'. "
              f"Return only the sentiment of the news headline.")
    
    # Check for chat template availability
    if hasattr(tokenizer, 'apply_chat_template') and tokenizer.chat_template:
        messages = [{"role": "user", "content": prompt}]
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    else:
        # Fallback for models without chat template (like base Mistral sometimes)
        inputs = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    
    with torch.no_grad():
        output = model.generate(
            inputs, 
            max_new_tokens=10, 
            temperature=0.01, 
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the new tokens
    res = tokenizer.decode(output[0][len(inputs[0]):], skip_special_tokens=True).lower()
    
    if 'positive' in res: return 2
    if 'negative' in res: return 0
    return 1 # neutral

def predict_finbert(text, model, tokenizer):
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    if prediction == 0: return 2 # Positive
    if prediction == 1: return 0 # Negative
    return 1 # Neutral

def predict_vader(text, analyzer):
    scores = analyzer.polarity_scores(str(text))
    compound = scores['compound']
    if compound >= 0.05: return 2
    if compound <= -0.05: return 0
    return 1

# --- ROBUST DATA LOADER ---
def load_datasets():
    path_lab = "/kaggle/input/fullda/Fin_Cleaned.csv"
    path_phr = "/kaggle/input/fullda/all-data.csv"

    print(f"\n🔍 Checking for datasets...")
    df_lab = pd.DataFrame()
    df_phr = pd.DataFrame()

    # Load Labelled Data (Headers Exist)
    if os.path.exists(path_lab):
        try:
            df_lab = pd.read_csv(path_lab, encoding='ISO-8859-1', on_bad_lines='skip')
            df_lab['label'] = df_lab['Final Status'].astype(str).str.lower().str.strip().map({'negative': 0, 'neutral': 1, 'positive': 2})
            df_lab = df_lab.rename(columns={'Headline': 'text'})
            df_lab = df_lab.dropna(subset=['label', 'text'])
            print(f"✅ Labelled Data: Loaded {len(df_lab)} samples.")
        except Exception as e:
            print(f"⚠️ Error loading Labelled Data: {e}")
    else:
        print(f"❌ Labelled Data NOT FOUND at {path_lab}")

    # Load Phrasebank (No Headers)
    if os.path.exists(path_phr):
        try:
            df_phr = pd.read_csv(path_phr, header=None, names=['label_str', 'text'], encoding='ISO-8859-1', on_bad_lines='skip')
            df_phr['label'] = df_phr['label_str'].astype(str).str.lower().str.strip().map({'negative': 0, 'neutral': 1, 'positive': 2})
            df_phr = df_phr.dropna(subset=['label', 'text'])
            print(f"✅ Phrasebank: Loaded {len(df_phr)} samples.")
        except Exception as e:
            print(f"⚠️ Error loading Phrasebank: {e}")
    else:
        print(f"❌ Phrasebank NOT FOUND at {path_phr}")

    return df_phr, df_lab

# --- MAIN LOOP ---
def run_evaluation():
    df_phr, df_lab = load_datasets()
    if df_phr.empty and df_lab.empty: return pd.DataFrame()
    
    final_results = []
    
    # Initialize VADER
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        analyzer = SentimentIntensityAnalyzer()
    except:
        subprocess.check_call(["pip", "install", "vaderSentiment"])
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        analyzer = SentimentIntensityAnalyzer()

    # Config for 4-bit loading
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    for m_cfg in MODELS_TO_TEST:
        print(f"\n--- 🚀 Starting {m_cfg['name']} ---")
        clean_memory()
        
        try:
            # 1. LOAD MODEL
            if m_cfg['type'] == 'generative':
                tokenizer = AutoTokenizer.from_pretrained(m_cfg['id'], token=True) # Uses logged-in token
                tokenizer.pad_token_id = tokenizer.eos_token_id 
                model = AutoModelForCausalLM.from_pretrained(
                    m_cfg['id'], 
                    quantization_config=bnb_config, 
                    device_map="auto",
                    token=True # Uses logged-in token
                )
                predict_func = lambda t: predict_generative(t, model, tokenizer)
                
            elif m_cfg['type'] == 'bert':
                tokenizer = AutoTokenizer.from_pretrained(m_cfg['id'])
                model = AutoModelForSequenceClassification.from_pretrained(m_cfg['id'], device_map="auto")
                predict_func = lambda t: predict_finbert(t, model, tokenizer)
                
            elif m_cfg['type'] == 'vader':
                predict_func = lambda t: predict_vader(t, analyzer)
                model = None

            results = {"Model": m_cfg['name']}
            
            # 2. EVALUATE
            if not df_lab.empty:
                y_true = df_lab['label'].values
                y_pred = [predict_func(t) for t in tqdm(df_lab['text'], desc="Labelled")]
                results["Acc (Labelled)"] = round(accuracy_score(y_true, y_pred), 4)
            else:
                results["Acc (Labelled)"] = 0.0

            if not df_phr.empty:
                y_true = df_phr['label'].values
                y_pred = [predict_func(t) for t in tqdm(df_phr['text'], desc="Phrasebank")]
                results["Acc (Phrasebank)"] = round(accuracy_score(y_true, y_pred), 4)
            else:
                results["Acc (Phrasebank)"] = 0.0
            
            # 3. SCORE
            if df_phr.empty: results["Final Score"] = results["Acc (Labelled)"]
            elif df_lab.empty: results["Final Score"] = results["Acc (Phrasebank)"]
            else: results["Final Score"] = (results["Acc (Labelled)"] + results["Acc (Phrasebank)"]) / 2
            
            results["Final Score"] = round(results["Final Score"], 4)
            final_results.append(results)
            print(f"📊 Result: {results['Final Score']}")

            if model: del model; del tokenizer
            clean_memory()

        except Exception as e:
            print(f"❌ FAILED {m_cfg['name']}: {str(e)}")
            clean_memory()
            continue
        
    return pd.DataFrame(final_results).sort_values("Final Score", ascending=False)

if __name__ == "__main__":
    df = run_evaluation()
    print("\n--- SCOREBOARD ---")
    if not df.empty:
        print(df.to_string(index=False))


🔍 Checking for datasets...
✅ Labelled Data: Loaded 400 samples.
✅ Phrasebank: Loaded 4846 samples.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.0 MB/s eta 0:00:00


PackageNotFoundError: No package metadata was found for bitsandbytes